In [2]:
pip install lightning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 828.5/828.5 kB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 73.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 832.4/832.4 kB 63.5 MB/s eta 0:00:00


In [4]:
# -*- coding: utf-8 -*-
import torch
import torch.nn as nn
import torch.nn.functional as F
from lightning.pytorch import LightningModule
from torchmetrics import Metric


# =====================================================
# Pixel MAE Metric (1920×1080 as in DGAZE)
# =====================================================
class PixelMAE(Metric):
    def __init__(self, screen_w=1920, screen_h=1080):
        super().__init__(dist_sync_on_step=False)
        self.add_state("sum_abs_err", default=torch.tensor(0.0), dist_reduce_fx="sum")
        self.add_state("total", default=torch.tensor(0), dist_reduce_fx="sum")
        self.screen_w = screen_w
        self.screen_h = screen_h

    def update(self, preds: torch.Tensor, target: torch.Tensor):
        scale = torch.tensor([self.screen_w, self.screen_h], device=preds.device)
        preds_px = preds * scale
        target_px = target * scale
        err = torch.abs(preds_px - target_px).sum(dim=1)  # L1 per sample
        self.sum_abs_err += err.sum()
        self.total += preds.size(0)

    def compute(self):
        return self.sum_abs_err / self.total


# =====================================================
# SimpleEyeEncoder
# =====================================================
class SimpleEyeEncoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool = nn.MaxPool2d(2)
        self.fc = nn.Linear(128, 512)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.adaptive_avg_pool2d(x, 1).view(x.size(0), -1)
        v = F.relu(self.fc(x))
        return v


# =====================================================
# Residual Dense Block
# =====================================================
class ResidualDenseBlock(nn.Module):
    def __init__(self, in_channels, out_channels=None):
        super().__init__()
        out_channels = out_channels or in_channels
        self.conv1 = nn.Conv2d(in_channels, out_channels, 3, padding=1)
        self.conv2 = nn.Conv2d(in_channels + out_channels, out_channels, 3, padding=1)
        self.conv3 = nn.Conv2d(in_channels + 2 * out_channels, out_channels, 3, padding=1)
        self.lff_conv = nn.Conv2d(in_channels + 3 * out_channels, out_channels, 1)

    def forward(self, x):
        c1 = F.relu(self.conv1(x))
        c2 = F.relu(self.conv2(torch.cat([x, c1], 1)))
        c3 = F.relu(self.conv3(torch.cat([x, c1, c2], 1)))
        lff = self.lff_conv(torch.cat([x, c1, c2, c3], 1))
        return lff + x


# =====================================================
# ResDenseEyeEncoder
# =====================================================
class ResDenseEyeEncoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.initial_conv = nn.Conv2d(3, 32, 3, padding=1)
        self.rdbs = nn.Sequential(
            ResidualDenseBlock(32),
            ResidualDenseBlock(32),
            ResidualDenseBlock(32)
        )
        self.global_fusion_conv = nn.Conv2d(32 * 3, 32, 1)
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(32, 512)

    def forward(self, x):
        x = F.relu(self.initial_conv(x))
        rdb1 = self.rdbs[0](x)
        rdb2 = self.rdbs[1](rdb1)
        rdb3 = self.rdbs[2](rdb2)
        gff_input = torch.cat([rdb1, rdb2, rdb3], 1)
        gff_output = self.global_fusion_conv(gff_input)
        final_output = x + gff_output
        v = self.pool(final_output).view(final_output.size(0), -1)
        v = F.relu(self.fc(v))
        return v


# =====================================================
# Landmarks Encoder
# =====================================================
class LandmarksEncoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(14, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU()
        )

    def forward(self, lm):
        return self.net(lm)


# =====================================================
# Attention module
# =====================================================
class AttentionModule(nn.Module):
    def __init__(self, feature_dim):
        super().__init__()
        self.attention = nn.Sequential(
            nn.Linear(feature_dim, feature_dim),
            nn.ReLU(),
            nn.Linear(feature_dim, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        w = self.attention(x)
        return x * w


# =====================================================
# Gaze Model (modular with encoder)
# =====================================================
class GazeModel(nn.Module):
    def __init__(self, EncoderCls):
        super().__init__()
        self.eye = EncoderCls()
        self.lm = LandmarksEncoder()
        self.eye_attention = AttentionModule(512)
        self.lm_attention = AttentionModule(128)
        self.fusion = nn.Sequential(
            nn.Linear(512 + 128, 256),
            nn.ReLU(),
            nn.Dropout(0.2)
        )
        self.regressor = nn.Sequential(
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64, 2)
        )

    def forward(self, left_eye, landmarks):
        v_eye = self.eye(left_eye)
        v_lm = self.lm(landmarks)
        v_eye = self.eye_attention(v_eye)
        v_lm = self.lm_attention(v_lm)
        fused = torch.cat([v_eye, v_lm], dim=1)
        fused = self.fusion(fused)
        coords = self.regressor(fused)
        return coords


# =====================================================
# Shared LightningModule
# =====================================================
class LitBase(LightningModule):
    def __init__(self,
                 model,
                 lr=1e-4,
                 weight_decay=1e-5,
                 loss_fn=nn.L1Loss(),
                 pixel_mae_train=PixelMAE(),
                 pixel_mae_val=PixelMAE(),
                 pixel_mae_test=PixelMAE(),
                 ):
        super().__init__()
        self.model           = model
        self.lr              = lr
        self.weight_decay    = weight_decay
        self.loss_fn         = loss_fn
        self.pixel_mae_train = pixel_mae_train
        self.pixel_mae_val   = pixel_mae_val
        self.pixel_mae_test  = pixel_mae_test

    def forward(self, left_eye, landmarks):
        return self.model(left_eye, landmarks)

    def training_step(self, batch, batch_idx):
        left_eye, landmarks, gaze_xy = batch
        preds = self(left_eye, landmarks)
        loss = self.loss_fn(preds, gaze_xy)
        self.pixel_mae_train.update(preds, gaze_xy)
        self.log("train/loss", loss, on_step=False, on_epoch=True)
        return loss

    def on_train_epoch_end(self):
        mae = self.pixel_mae_train.compute()
        self.log("train/mae_pixels", mae, prog_bar=True)
        self.pixel_mae_train.reset()

    def validation_step(self, batch, batch_idx):
        left_eye, landmarks, gaze_xy = batch
        preds = self(left_eye, landmarks)
        loss = self.loss_fn(preds, gaze_xy)
        self.pixel_mae_val.update(preds, gaze_xy)
        self.log("val/loss", loss, prog_bar=True)
        return loss

    def on_validation_epoch_end(self):
        mae = self.pixel_mae_val.compute()
        self.log("val/mae_pixels", mae, prog_bar=True)
        self.pixel_mae_val.reset()

    def test_step(self, batch, batch_idx):
        left_eye, landmarks, gaze_xy = batch
        preds = self(left_eye, landmarks)
        self.pixel_mae_test.update(preds, gaze_xy)

    def on_test_epoch_end(self):
        mae = self.pixel_mae_test.compute()
        self.log("test/mae_pixels", mae, prog_bar=True)
        self.pixel_mae_test.reset()

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(
            self.parameters(), lr=self.lr, weight_decay=self.weight_decay
        )
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode="min", factor=0.1, patience=12, min_lr=1e-6
        )
        return {
            "optimizer": optimizer,
            "lr_scheduler": {"scheduler": scheduler, "monitor": "val/loss"},
        }


# =====================================================
# Convenience Wrappers
# =====================================================
class SimpleLitModel(LitBase):
    def __init__(self, *args, **kwargs):
        super().__init__(GazeModel(SimpleEyeEncoder), *args, **kwargs)


class ResDenseLitModel(LitBase):
    def __init__(self, *args, **kwargs):
        super().__init__(GazeModel(ResDenseEyeEncoder), *args, **kwargs)
